# Microsoft SEAL : CKKS scheme with C wrapper

In this example, I compute the following formula, in which $ x $ is encrypted by homomorphic encryption, which can only (HE) library, Microsoft SEAL.

$ 3.14159265 x^3 + 0.5 $

for $ x=0.0, 1.1, 2.2, 3.3 $

Here I use CKKS (Cheon-Kim-Kim-Song) scheme for homomorphic encryption, which can operate the encrypted real and complex numbers.

To set up the required software, see the [previous exercise](./01-seal-python-bfv-with-c-wrapper.ipynb). (Here I use Microsoft SEAL and EVA compiler in Python.)

*back to [Readme](https://github.com/tsmatz/homomorphic-encryption-microsoft-seal/)*

In [1]:
import numpy as np
import math
import base64

In [2]:
import ctypes
seal_lib = ctypes.CDLL("./SEAL/build/lib/libsealc.so.3.7.2")

In [3]:
#
# Define native function's arguments
#
seal_lib.EncParams_Create1.argtypes = [ ctypes.c_byte, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.EncParams_SetPolyModulusDegree.argtypes = [ ctypes.c_void_p, ctypes.c_ulonglong ]
seal_lib.EncParams_SetCoeffModulus.argtypes = [ ctypes.c_void_p, ctypes.c_ulonglong, ctypes.POINTER(ctypes.c_ulonglong) ]
seal_lib.CoeffModulus_Create.argtypes = [ ctypes.c_ulonglong, ctypes.c_ulonglong, ctypes.POINTER(ctypes.c_int), ctypes.POINTER(ctypes.c_ulonglong) ]
seal_lib.SEALContext_Create.argtypes = [ ctypes.c_void_p, ctypes.c_bool, ctypes.c_int, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.SEALContext_FirstParmsId.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_ulonglong) ]
seal_lib.KeyGenerator_Create1.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.KeyGenerator_SecretKey.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.KeyGenerator_CreatePublicKey.argtypes = [ ctypes.c_void_p, ctypes.c_bool, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.KeyGenerator_CreateRelinKeys.argtypes = [ ctypes.c_void_p, ctypes.c_bool, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.CKKSEncoder_Create.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.CKKSEncoder_SlotCount.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_ulonglong) ]
seal_lib.CKKSEncoder_Encode1.argtypes = [ ctypes.c_void_p, ctypes.c_ulonglong, ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_ulonglong), ctypes.c_double, ctypes.c_void_p, ctypes.c_void_p ]
seal_lib.CKKSEncoder_Encode3.argtypes = [ ctypes.c_void_p, ctypes.c_double, ctypes.POINTER(ctypes.c_ulonglong), ctypes.c_double, ctypes.c_void_p, ctypes.c_void_p ]
seal_lib.CKKSEncoder_Decode1.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.POINTER(ctypes.c_ulonglong), ctypes.POINTER(ctypes.c_double), ctypes.c_void_p ]
seal_lib.Plaintext_Create1.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.Encryptor_Create.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.Encryptor_Encrypt.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p ]
seal_lib.Ciphertext_Create1.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.Ciphertext_SaveSize.argtypes = [ ctypes.c_void_p, ctypes.c_ubyte, ctypes.POINTER(ctypes.c_ulonglong) ]
seal_lib.Ciphertext_Save.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_ubyte), ctypes.c_ulonglong, ctypes.c_ubyte, ctypes.POINTER(ctypes.c_ulonglong) ]
seal_lib.Ciphertext_Scale.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_double) ]
seal_lib.Ciphertext_SetScale.argtypes = [ ctypes.c_void_p, ctypes.c_double ]
seal_lib.Ciphertext_ParmsId.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_ulonglong) ]
seal_lib.Serialization_ComprModeDefault.argtypes = [ ctypes.POINTER(ctypes.c_ubyte) ]
seal_lib.Evaluator_Create.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.Evaluator_Square.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p ]
seal_lib.Evaluator_RescaleToNext.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p ]
seal_lib.Evaluator_Multiply.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p ]
seal_lib.Evaluator_MultiplyPlain.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p ]
seal_lib.Evaluator_AddPlain.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p ]
seal_lib.Evaluator_ModSwitchTo2.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.POINTER(ctypes.c_ulonglong), ctypes.c_void_p ]
seal_lib.Decryptor_Create.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.Decryptor_Decrypt.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p ]

In [4]:
#
# Define error handling
#
def HandleError(error):
    if (error != 0):
        raise OSError('Failed with result: %s' %hex(error))

In [5]:
#
# Create encryption parameter (CKKS)
#
ptr_encparm = ctypes.c_void_p()
HandleError(seal_lib.EncParams_Create1(
    ctypes.c_byte(0x02), # 0x02 means CKKS
    ctypes.byref(ptr_encparm)
))

In [6]:
#
# Set encryption parameter details
#

# 1. Set PolyModulusDegree
HandleError(seal_lib.EncParams_SetPolyModulusDegree(
    ptr_encparm,
    ctypes.c_ulonglong(8192)
))
# 2. Create CoeffModulus
# (Here I create 5 40-bit prime numbers.)
coeff_size_arr = (ctypes.c_int * 5)()
coeff_size_arr[0] = 40
coeff_size_arr[1] = 40
coeff_size_arr[2] = 40
coeff_size_arr[3] = 40
coeff_size_arr[4] = 40
coeff_arr = (ctypes.c_ulong * 5)()
HandleError(seal_lib.CoeffModulus_Create(
    ctypes.c_ulonglong(8192),
    ctypes.c_ulonglong(5),
    ctypes.cast(coeff_size_arr, ctypes.POINTER(ctypes.c_int)),
    ctypes.cast(coeff_arr, ctypes.POINTER(ctypes.c_ulong))
))
# for i in coeff_arr: print(i) # uncomment to check values
# 3. Set CoeffModulus
HandleError(seal_lib.EncParams_SetCoeffModulus(
    ptr_encparm,
    ctypes.c_ulonglong(5),
    ctypes.cast(coeff_arr, ctypes.POINTER(ctypes.c_ulong))
))

In [7]:
#
# Create SEAL context
#

# Create
ptr_context = ctypes.c_void_p()
HandleError(seal_lib.SEALContext_Create(
    ptr_encparm,
    ctypes.c_bool(True),
    ctypes.c_int(128),
    ctypes.byref(ptr_context)
))
# Get first parms id
parms_id = ctypes.c_ulonglong()
HandleError(seal_lib.SEALContext_FirstParmsId(
    ptr_context,
    ctypes.byref(parms_id)
))

In [8]:
#
# Create keys for encryption, decryption, and relinearization
#

# Create key generator
ptr_key_generator = ctypes.c_void_p()
HandleError(seal_lib.KeyGenerator_Create1(
    ptr_context,
    ctypes.byref(ptr_key_generator)
))
# Get secret key used for decryption
# (Use SecretKey_Data when you show secret key text)
ptr_secret_key = ctypes.c_void_p()
HandleError(seal_lib.KeyGenerator_SecretKey(
    ptr_key_generator,
    ctypes.byref(ptr_secret_key)
))
# Create public key used for encryption
ptr_public_key = ctypes.c_void_p()
HandleError(seal_lib.KeyGenerator_CreatePublicKey(
    ptr_key_generator,
    ctypes.c_bool(False),
    ctypes.byref(ptr_public_key)
))
# Create relinearization key used for relinearization
ptr_relin_key = ctypes.c_void_p()
HandleError(seal_lib.KeyGenerator_CreateRelinKeys(
    ptr_key_generator,
    ctypes.c_bool(False),
    ctypes.byref(ptr_relin_key)
))

In [9]:
#
# Create CKKS encoder
#

# Create CKKS encoder
ptr_encoder = ctypes.c_void_p()
HandleError(seal_lib.CKKSEncoder_Create(
    ptr_context,
    ctypes.byref(ptr_encoder)
))
# In CKKS, the number of slots is PolyModulusDegree / 2 and each slot encodes one number.
# In encryption, the encoder will implicitly pad it with zeros to full size, PolyModulusDegree / 2.
slot_size = ctypes.c_ulonglong()
HandleError(seal_lib.CKKSEncoder_SlotCount(
    ptr_encoder,
    ctypes.byref(slot_size)
))
print("Slot size is {}".format(slot_size.value))

Slot size is 4096


In [10]:
#
# Create plaintexts for PI (3.14159265) and 0.5
# with CKKS encoder
# (Encodes these floating-point values to every slot)
#

# Scale S = 2^40
# (If ciphertexts have scale S, they'll have scale S^2 after multiplication.)
scale = pow(2.0, 40)
# Create plain text for PI
ptr_plain_pi = ctypes.c_void_p()
HandleError(seal_lib.Plaintext_Create1(
    None,
    ctypes.byref(ptr_plain_pi)
))
# CKKS encode for PI
HandleError(seal_lib.CKKSEncoder_Encode3(
    ptr_encoder,
    ctypes.c_double(3.14159265),
    ctypes.byref(parms_id),
    ctypes.c_double(scale),
    ptr_plain_pi,
    None
))
# Create plain text for 0.5
ptr_plain_05 = ctypes.c_void_p()
HandleError(seal_lib.Plaintext_Create1(
    None,
    ctypes.byref(ptr_plain_05)
))
# CKKS encode for 0.5
HandleError(seal_lib.CKKSEncoder_Encode3(
    ptr_encoder,
    ctypes.c_double(0.5),
    ctypes.byref(parms_id),
    ctypes.c_double(scale),
    ptr_plain_05,
    None
))

In [11]:
#
# Create encryption for x (= 0.0, 1.1, 2.2, 3.3)
# with CKKS encoder
# (Other slots will be padded with zeros)
#

# Create plain text for x
ptr_plain_x = ctypes.c_void_p()
HandleError(seal_lib.Plaintext_Create1(
    None,
    ctypes.byref(ptr_plain_x)
))
# CKKS encode for x
x_arr = (ctypes.c_double * 4)()
x_arr[0] = 0.0
x_arr[1] = 1.1
x_arr[2] = 2.2
x_arr[3] = 3.3
HandleError(seal_lib.CKKSEncoder_Encode1(
    ptr_encoder,
    ctypes.c_ulonglong(4),
    ctypes.cast(x_arr, ctypes.POINTER(ctypes.c_double)),
    ctypes.byref(parms_id),
    ctypes.c_double(scale),
    ptr_plain_x,
    None
))
# Create cipher text for x
ptr_cipher_x = ctypes.c_void_p()
HandleError(seal_lib.Ciphertext_Create1(
    None,
    ctypes.byref(ptr_cipher_x)
))
# Create encryptor
ptr_encryptor = ctypes.c_void_p()
HandleError(seal_lib.Encryptor_Create(
    ptr_context,
    ptr_public_key,
    None,
    ctypes.byref(ptr_encryptor)
))
# Encrypt for x
HandleError(seal_lib.Encryptor_Encrypt(
    ptr_encryptor,
    ptr_plain_x,
    ptr_cipher_x,
    None
))

In [12]:
#
# Encode and output cipher text for x
#
# (Here I only show base64 encoded string,
#  but this will be needed for passing a cipher data on network.)
#

# Get default compare mode
compr_mode = ctypes.c_ubyte()
HandleError(seal_lib.Serialization_ComprModeDefault(
    ctypes.byref(compr_mode)
))
# Get save size
save_size = ctypes.c_ulonglong()
HandleError(seal_lib.Ciphertext_SaveSize(
    ptr_cipher_x,
    compr_mode,
    ctypes.byref(save_size)
))
# Write encrypted bytes to buffer
save_size_output = ctypes.c_ulonglong()
byte_arr_x = (ctypes.c_ubyte * save_size.value)()
HandleError(seal_lib.Ciphertext_Save(    
    ptr_cipher_x,
    ctypes.cast(byte_arr_x, ctypes.POINTER(ctypes.c_ubyte)),
    save_size,
    compr_mode,
    ctypes.byref(save_size_output)
))
# Base64 encode
byte_nparr_x = np.array(byte_arr_x[:save_size_output.value], dtype = ctypes.c_ubyte)
b64_x = base64.b64encode(byte_nparr_x)
print("********** Base64 encoded x **********")
print(b64_x)

********** Base64 encoded x **********
b'...'


In [13]:
#
# Compute x^2
#

# Create Evaluator
ptr_evaluator = ctypes.c_void_p()
HandleError(seal_lib.Evaluator_Create(
    ptr_context,
    ctypes.byref(ptr_evaluator)
))
# Create cipher text for result
ptr_cipher_res1 = ctypes.c_void_p()
HandleError(seal_lib.Ciphertext_Create1(
    None,
    ctypes.byref(ptr_cipher_res1)
))
# Square x
HandleError(seal_lib.Evaluator_Square(
    ptr_evaluator,
    ptr_cipher_x,
    ptr_cipher_res1,
    None
))
# Get size and scale of cipher
cipher_size = ctypes.c_ulonglong()
HandleError(seal_lib.Ciphertext_Size(
    ptr_cipher_res1,
    ctypes.byref(cipher_size)
))
cipher_scale = ctypes.c_double()
HandleError(seal_lib.Ciphertext_Scale(
    ptr_cipher_res1,
    ctypes.byref(cipher_scale)
))
print("Size  (before) : {}".format(cipher_size.value))
print("Scale (before) : {}".format(math.log(cipher_scale.value, 2)))
# Relinearize to reduce the size of a cipher 
HandleError(seal_lib.Evaluator_Relinearize(
    ptr_evaluator,
    ptr_cipher_res1,
    ptr_relin_key,
    ptr_cipher_res1,
    None
))
# Rescale to set appropriate scale
HandleError(seal_lib.Evaluator_RescaleToNext(
    ptr_evaluator,
    ptr_cipher_res1,
    ptr_cipher_res1,
    None
))
# Get size and scale of cipher
HandleError(seal_lib.Ciphertext_Size(
    ptr_cipher_res1,
    ctypes.byref(cipher_size)
))
HandleError(seal_lib.Ciphertext_Scale(
    ptr_cipher_res1,
    ctypes.byref(cipher_scale)
))
print("Size  (after) : {}".format(cipher_size.value))
print("Scale (after) : {}".format(math.log(cipher_scale.value, 2)))

Size  (before) : 3
Scale (before) : 80.0
Size  (after) : 2
Scale (after) : 40.00000096740143


In [14]:
#
# Compute 3.14159265 x
#

# Create cipher for the result of 3.14159265 x
ptr_cipher_res2 = ctypes.c_void_p()
HandleError(seal_lib.Ciphertext_Create1(
    None,
    ctypes.byref(ptr_cipher_res2)
))
# Compute 3.14159265 x
HandleError(seal_lib.Evaluator_MultiplyPlain(
    ptr_evaluator,
    ptr_cipher_x,
    ptr_plain_pi,
    ptr_cipher_res2,
    None
))
# Get scale of cipher
HandleError(seal_lib.Ciphertext_Scale(
    ptr_cipher_res2,
    ctypes.byref(cipher_scale)
))
print("Scale (before) : {}".format(math.log(cipher_scale.value, 2)))
# Rescale to set appropriate scale
HandleError(seal_lib.Evaluator_RescaleToNext(
    ptr_evaluator,
    ptr_cipher_res2,
    ptr_cipher_res2,
    None
))
# Get scale of cipher
HandleError(seal_lib.Ciphertext_Scale(
    ptr_cipher_res2,
    ctypes.byref(cipher_scale)
))
print("Scale (before) : {}".format(math.log(cipher_scale.value, 2)))

Scale (before) : 80.0
Scale (before) : 40.00000096740143


In [15]:
#
# Compute 3.14159265 x^3
# by mutiplying x^2 and 3.14159265 x
#

# Multiply x^2 and 3.14159265 x
HandleError(seal_lib.Evaluator_Multiply(
    ptr_evaluator,
    ptr_cipher_res1,
    ptr_cipher_res2,
    ptr_cipher_res2,
    None
))
# Get size and scale of cipher
HandleError(seal_lib.Ciphertext_Size(
    ptr_cipher_res2,
    ctypes.byref(cipher_size)
))
HandleError(seal_lib.Ciphertext_Scale(
    ptr_cipher_res2,
    ctypes.byref(cipher_scale)
))
print("Size  (before) : {}".format(cipher_size.value))
print("Scale (before) : {}".format(math.log(cipher_scale.value, 2)))
# Relinearize to reduce the size of a cipher 
HandleError(seal_lib.Evaluator_Relinearize(
    ptr_evaluator,
    ptr_cipher_res2,
    ptr_relin_key,
    ptr_cipher_res2,
    None
))
# Rescale to set appropriate scale
HandleError(seal_lib.Evaluator_RescaleToNext(
    ptr_evaluator,
    ptr_cipher_res2,
    ptr_cipher_res2,
    None
))
# Get size and scale of cipher
HandleError(seal_lib.Ciphertext_Size(
    ptr_cipher_res2,
    ctypes.byref(cipher_size)
))
HandleError(seal_lib.Ciphertext_Scale(
    ptr_cipher_res2,
    ctypes.byref(cipher_scale)
))
print("Size  (after) : {}".format(cipher_size.value))
print("Scale (after) : {}".format(math.log(cipher_scale.value, 2)))

Size  (before) : 3
Scale (before) : 80.00000193480285
Size  (after) : 2
Scale (after) : 40.00000298819566


In [16]:
#
# Compute 3.14159265 x^3 + 0.5
#

# Normalize scale to 2^40, before adding plaintext 0.5
HandleError(seal_lib.Ciphertext_SetScale(
    ptr_cipher_res2,
    ctypes.c_double(pow(2.0, 40))
))
# Get parms id
cipher_parms_id = ctypes.c_ulonglong()
HandleError(seal_lib.Ciphertext_ParmsId(
    ptr_cipher_res2,
    ctypes.byref(cipher_parms_id)
))
# Normalize parameters, before adding plaintext 0.5
HandleError(seal_lib.Evaluator_ModSwitchTo2(
    ptr_evaluator,
    ptr_plain_05,
    ctypes.byref(cipher_parms_id),
    ptr_plain_05
))
# Compute 3.14159265 x^3 + 0.5
HandleError(seal_lib.Evaluator_AddPlain(
    ptr_evaluator,
    ptr_cipher_res2,
    ptr_plain_05,
    ptr_cipher_res2
))

In [17]:
#
# Decrypt result
#

# Create Decryptor
ptr_decryptor = ctypes.c_void_p()
HandleError(seal_lib.Decryptor_Create(
    ptr_context,
    ptr_secret_key,
    ctypes.byref(ptr_decryptor)
))
# Create plain text for result
ptr_plain_res = ctypes.c_void_p()
HandleError(seal_lib.Plaintext_Create1(
    None,
    ctypes.byref(ptr_plain_res)
))
# Decrypt result
HandleError(seal_lib.Decryptor_Decrypt(
    ptr_decryptor,
    ptr_cipher_res2,
    ptr_plain_res
))
# Decode result with CKKS encoder
res_arr = (ctypes.c_double * slot_size.value)()
HandleError(seal_lib.CKKSEncoder_Decode1(
    ptr_encoder,
    ptr_plain_res,
    ctypes.byref(ctypes.c_ulonglong(0)),
    ctypes.cast(res_arr, ctypes.POINTER(ctypes.c_double)),
    None
))
# Output results
print("********** Result is **********")
for i in range(4):
    print(res_arr[i])

********** Result is **********
0.5000000006696688
4.68146842068375
33.951747763151246
113.39964892362529
